In [1]:
#!pip install deepface
!pip install tqdm

In [2]:
import os
import sys
from deepface import DeepFace
from PIL import Image, ImageDraw
from IPython.display import display
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from tqdm import tqdm 

sys.path.append('aux_scripts.py')

2025-01-13 18:38:06.776809: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 18:38:06.897075: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-13 18:38:06.984877: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736789887.071551  101318 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736789887.096371  101318 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-13 18:38:07.317923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Define the folder and image name

In [3]:
folder_name = "../banana/"
processed_images = []

for image_name in tqdm(os.listdir(folder_name), desc="Processing Images", unit="image"):    # Only process if the file is an image (e.g., PNG, JPG)
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        image_path = os.path.join(folder_name, image_name)
        image = Image.open(image_path)
        image_draw = Image.open(image_path)

        # Analyze the image
        analysis = DeepFace.analyze(img_path=image_path, detector_backend='retinaface', enforce_detection=False)
        
        # Draw bounding boxes for each face detected
        draw = ImageDraw.Draw(image_draw)
        for idx, item in enumerate(analysis):
            region = item['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            offset_x = 0.2
            offset_y = 0.15

            # Crop the detected face
            cropped_face = image.crop((x-((w*offset_x)/2), y-((h*offset_y)/2), x + w + w*offset_x, y + h + h*offset_y))
            img = cropped_face.resize((299, 299))
            img = img_to_array(img) / 255.0  # Normalize pixel values
            img = np.expand_dims(img, axis=0)  # Add batch dimension

            processed_images.append(img)


W0000 00:00:1736789890.216427  101318 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Action: age:  25%|██▌       | 1/4 [00:00<00:00,  5.62it/s]    2025-01-13 18:38:18.279711: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-13 18:38:18.375590: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-13 18:38:18.467588: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-01-13 18:38:19.791927: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 41104179

In [4]:
import os
import shutil
from IPython.display import display
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import numpy as np

model = tf.keras.models.load_model('mask_detector_model_3classes_better_wrong.h5')

for img in processed_images:
    # Make prediction
    prediction = model.predict(img)  # Prediction for the image
    predicted_class = np.argmax(prediction)  # Get class index with highest probability
    label = class_labels[predicted_class]  # Map index to class label

    # Display the image (using PIL for display)
    img_display = Image.fromarray((img[0] * 255).astype(np.uint8))  # Convert back to image format
    img_display.show()

    # Print the prediction
    print(f"Prediction: {label} (Probabilities: {prediction[0]})")


TypeError: Error when deserializing class 'InputLayer' using config={'batch_shape': [None, 299, 299, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer_1'}.

Exception encountered: Unrecognized keyword arguments: ['batch_shape']